<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# _*Working with the IBMQ provider*_ 

The latest version of this notebook is available on https://github.com/qiskit/qiskit-tutorial.

***
#### Contributors
Jay Gambetta[1], Diego Moreda[1], Ali Javadi[1], Eric Winston[1], Joe Hellmers, and Anna Phan[1]

#### Affiliations 
*[1] IBM Q*

In Qiskit we have an base interface for backends and jobs that will be useful for running circuits and extending to third-party backends. In this tutorial, we will review the core components of Qiskit’s base backend framework, using the IBM Q provider as an example.

The interface has three parts: the provider, the backend, and the job:

- provider: accesses backends and provides backend objects
- backend: runs the quantum circuit
- job: keeps track of the submitted job

## The IBMQ Provider

The IBMQ Provider is an entity that provides access to a group of different backends (for example, backends available through IBM Q Experience or IBM Q Network).

The IBMQ provider inherits from BaseProvider and implements the methods:

    backends(): returns all backend objects known to the provider.
    get_backend(name): returns the named backend.

The IBM Q provider has some extra functions for handling administrative tasks. The credentials can be saved to disk or user in a session and never saved. 

    enable_account(token, url): adds the account to current session
    disable_accounts(**kwargs): remove from account current session
    save_account(token, url): save the account to disk
    delete_accounts(**kwargs): delete the account or accounts from disk
    load_accounts(**kwargs): load the account or accounts into session
    active_accounts(): list all accounts active in this session
    stored_accounts(): list all accounts saved to disk

In [1]:
from qiskit import IBMQ 

IBMQ.backends()

[]

Here we see that there are no backends as no accounts have been loaded but we have some stored on the computer.

Start fresh. delete any accounts on disk. If no accounts it will error

In [2]:
IBMQ.delete_accounts()

verify that there are no accounts stored now

In [3]:
IBMQ.stored_accounts()

[]

To demonstrate that we can load multiple accounts into the IBMQ provider here we use a file `Qconfig_IBMQ_foo.py` which has variables

```python
APItoken = 'MY_API_TOKEN'
URL = 'THE_URL'
```

For the IBM Q experience the URL is not needed and is loaded by default. 

For the IBM Q Netwrok the url found on your q-console account page.

In [4]:
import sys
sys.path.append("../../")
import Qconfig_IBMQ_network
import Qconfig_IBMQ_experience

Enable an account (useful for one-off use, or if you don't want to save to disk)

In [5]:
IBMQ.enable_account(Qconfig_IBMQ_experience.APItoken)

To see that accounts enabled for use

In [6]:
# uncomment to print to screen (it will show your token and url)
# IBMQ.active_accounts()

and its backends are available

In [7]:
IBMQ.backends()

[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmqx5') from IBMQ()>,
 <IBMQBackend('ibmqx2') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>,
 <IBMQBackend('ibmq_qasm_simulator') from IBMQ()>]

Disable that account (so we go back to no accounts active)

In [8]:
IBMQ.disable_accounts(token=Qconfig_IBMQ_experience.APItoken)

Now no backends are available

In [9]:
IBMQ.backends()

[]

Save two accounts: a public (IBM Q experience) and a premium (IBM Q network)

In [10]:
IBMQ.save_account(Qconfig_IBMQ_experience.APItoken)
IBMQ.save_account(Qconfig_IBMQ_network.APItoken, Qconfig_IBMQ_network.url)

Now they should show up as present on disk

In [11]:
# uncomment to print to screen (it will show your token and url)
# IBMQ.stored_accounts()

but no account active in current session yet

In [12]:
IBMQ.active_accounts()

[]

so IBMQ can't see any backends yet

In [13]:
IBMQ.backends()

[]

now load up every account stored to disk

In [14]:
IBMQ.load_accounts()

backends from two different accounts available for use

In [15]:
IBMQ.backends()

[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmqx5') from IBMQ()>,
 <IBMQBackend('ibmqx2') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>,
 <IBMQBackend('ibmq_qasm_simulator') from IBMQ()>,
 <IBMQBackend('ibmq_20_tokyo') from IBMQ(ibm-q-internal, qiskit, qiskit-terra)>,
 <IBMQBackend('ibmq_qasm_simulator') from IBMQ(ibm-q-internal, qiskit, qiskit-terra)>]

now if you want to work with backends of a single account,
you can do so via backend filtering

In [16]:
IBMQ.backends(url='https://quantumexperience.ng.bluemix.net/api')

[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmqx5') from IBMQ()>,
 <IBMQBackend('ibmqx2') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>,
 <IBMQBackend('ibmq_qasm_simulator') from IBMQ()>]

but you can also just disable account in the current session

In [17]:
IBMQ.disable_accounts(url='https://quantumexperience.ng.bluemix.net/api')

so now only one account is active

In [18]:
# uncomment to print to screen (it will show your token and url)
# IBMQ.active_accounts()

and only that account's backends are available

In [19]:
IBMQ.backends()

[<IBMQBackend('ibmq_20_tokyo') from IBMQ(ibm-q-internal, qiskit, qiskit-terra)>,
 <IBMQBackend('ibmq_qasm_simulator') from IBMQ(ibm-q-internal, qiskit, qiskit-terra)>]

or from the start just load up that account you're interested in

In [20]:
IBMQ.disable_accounts()
IBMQ.load_accounts(url='https://quantumexperience.ng.bluemix.net/api')
IBMQ.backends()

[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmqx5') from IBMQ()>,
 <IBMQBackend('ibmqx2') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>,
 <IBMQBackend('ibmq_qasm_simulator') from IBMQ()>]

## Filtering the backends

You may also optionally filter the set of returned backends, by passing arguments that query the backend's `configuration` or `status` or `properties`.

The filters are passed by conditions and for more general you can make advanced functions using the lambda function.

As a first example: only return currently operational devices

In [22]:
IBMQ.backends(operational=True, simulator=False)

[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>]

only return backends that are real devices and have more than 10 qubits and are operational

In [28]:
IBMQ.backends(filters=lambda x: x.configuration()['n_qubits'] > 10 and 
              not x.configuration()['simulator'] and x.status()['operational']==True)

[<IBMQBackend('ibmq_16_melbourne') from IBMQ()>]

Filter: show the least busy device (in terms of pending jobs in the queue)

In [29]:
from qiskit.backends.ibmq import least_busy
least_busy(IBMQ.backends(simulator=False))

<IBMQBackend('ibmq_16_melbourne') from IBMQ()>

The above filters can be combined as desired.

If you just want to get an instance of a particular backend, you can use the `get_backend()` method of either Aer or IBMQ.

In [31]:
IBMQ.get_backend('ibmq_16_melbourne')

<IBMQBackend('ibmq_16_melbourne') from IBMQ()>

A backend's name is accesible through:

In [32]:
backend = IBMQ.get_backend('ibmq_16_melbourne')
backend.name()

'ibmq_16_melbourne'

## The backend

The next part is the backend instances. These backends must support the following functions:

```
configuration - gets the configuration of the backend.
```
```
properties -  gets the properties of the backend (currently calibrations and parameters, but will be combined).
```
```
status - gets the status of the backend.
```
```
jobs  -  returns a list of previous jobs executed by this user on this backend.
```
```
run  - runs a qobj on the backend.
```


Lets start with the `status()`:

    `operational` lets us know that the backend is taking jobs

    `pending_jobs` lets you know how many jobs are in the queue

In [ ]:
for backend in IBMQ.backends():
    pprint(backend.status())

The next is `configuration`

In [ ]:
for backend in IBMQ.backends():
    pprint(backend.configuration())

In [ ]:
for backend in IBMQ.backends():
    pprint(backend.properties())

### The Job object

To get a job first we must get a `qobj` (quantum object code). This is a list of circuits compiled to a backend. To get this we have made a `compile` funtion that converts an array of circuits into a `qobj`. The reason this is an array is for current hardware lists of circuits are loaded onto the equipment at once and it is much more efficent to run this in a batch mode. The methods of the jobs object are:

```
status - returns the status.
```
```
done - returns a bool true if done.
```
```
job_id  - gets the job_id (not supported in the local provider yet).
```
```
cancel - cancels the job (not supported on remote-public or local. Only supported on IBM Q premium devices).
```
```
result - gets the results from the circuit run.
```

In [ ]:
qr = QuantumRegister(3)
cr = ClassicalRegister(3)
circuit = QuantumCircuit(qr, cr)
circuit.x(qr[0])
circuit.x(qr[1])
circuit.ccx(qr[0], qr[1], qr[2])
circuit.cx(qr[0], qr[1])
circuit.measure(qr, cr)

In [ ]:
my_backend = Aer.get_backend('qasm_simulator')

In [ ]:
qobj = compile(circuit, backend=my_backend, shots=1024)
job = my_backend.run(qobj)

In [ ]:
result = job.result()
counts = result.get_counts()
print(counts)

In [ ]:
from qiskit.backends.ibmq import least_busy
device = least_busy(IBMQ.backends(simulator=False))

In [ ]:
%%qiskit_job_status
qobj = compile(circuit, backend=device, shots=1024)
job = device.run(qobj)

In [ ]:
job.job_id()

In [ ]:
result = job.result()
counts = result.get_counts()
print(counts)

In [ ]:
plot_histogram(counts)